# After  #2 demo, now try to pull diabetic patients record

Advice: As we move into more complex SQL queries, for efficiency purpose, I recommend to practise SQL code in pgAdmin first before load into Jupyter

## Reference

* Article: "Improving Patient Cohort Identification Using Natural Language Processing" 10 Sep. 2016, https://link.springer.com/chapter/10.1007/978-3-319-43742-2_28. Accessed 25 Nov. 2018.
* Git repository: https://github.com/MIT-LCP/critical-data-book
* To understand Mimic III tables: https://mimic.physionet.org/mimictables/
* To understand ICD_9 codes: http://www.icd9data.com/
* SQL query: https://github.com/MIT-LCP/critical-data-book/tree/master/part_iii/chapter_28

## Prepare database

Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import getpass
%matplotlib inline
plt.style.use('ggplot')

Create database connection, you can load your password here just for practise but not recommend to save into repository

In [2]:
user = 'postgres'
password = 'postgres'      # note if you want to set password for offline pracise would be fine, but don't post to repository
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'        # set to your defined schema name, I use mimiciii here

Connect to the database (execute again if lost connection)

In [3]:
con = psycopg2.connect(dbname=dbname, user=user, host=host, password=password)
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))          # this is compulsory step to find your database
print('Thread opened' if not cur.closed else 'Thread closed')

Thread opened


In [10]:
# If need password sensitive, use below code
# con = psycopg2.connect(dbname=dbname, user=user, host=host, 
#                        password=getpass.getpass(prompt='Password:'.format(user)))
# cur = con.cursor()
# cur.execute('SET search_path to {}'.format(schema))          # this is compulsory step to find your database
# print('connected' if not cur.connection.closed else 'not connected')

Password:········
connected


## Query using Structured data

Structured data means all records are categorized in the table so we can just classify them<br>
Unstructured data means records are in the text note area and need text mining to capture them

### First, validate whether the table we understand matches what article describes: 

* The unstructured clinical notes include:
    - discharge summaries
    - nursing progress notes
    - physician notes
    - electrocardiogram (ECG) reports
    - echocardiogram reports
    - and radiology reports
* We excluded clinical notes that were related to any imaging results (ECG_Report, Echo_Report, and Radiology_Report). 
* We extracted notes from MIMIC-III with the following data elements: 
    - patient identification number (SUBJECT_ID), 
    - hospital admission identification number (HADM_IDs), 
    - intensive care unit stay identification number (ICUSTAY_ID), 
    - note type, 
    - note date/time, 
    - and note text.

### It's our first time query Mimic III db, will practise a few queries to validate the procedure is working well.

The tables that are used in the queries:
* admissions: include subject_id, all patients
* diagnoses_icd: include icd9_code, subject_id, patients under diagnosis (covered all patients)
* patients: include subject_id, dob (covered all patients)
* PROCEDURES_ICD: include subject_id, those who were under procedures (subset of all patients)

In [4]:
# Find list of categories under noteevents table
query = \
"""
select distinct(category)
from noteevents;
"""
data = pd.read_sql_query(query, con)
data

,category
0,Consult
1,Social Work
2,ECG
3,Case Management
4,Nutrition
5,Radiology
6,Discharge summary
7,Pharmacy
8,Respiratory
9,Nursing/other


In [12]:
# Discharge summaries
query = \
"""
select count(distinct(subject_id))
from noteevents
where category like 'Discharge summary%'; 
"""
data = pd.read_sql_query(query, con)
data

,count
0,41127


In [13]:
# Nursing/Nursing others
query = \
"""
select count(distinct(subject_id))
from noteevents
where category like 'Nursing%'; 
"""
data = pd.read_sql_query(query, con)
data

,count
0,36583


In [14]:
# Physician notes
query = \
"""
select count(distinct(subject_id))
from noteevents
where category like 'Physician%'; 
"""
data = pd.read_sql_query(query, con)
data

,count
0,7623


In [15]:
# ECG reports
query = \
"""
select count(distinct(subject_id))
from noteevents
where category like 'ECG%'; 
"""
data = pd.read_sql_query(query, con)
data

,count
0,35366


In [16]:
# Echocardiogram reports
query = \
"""
select count(distinct(subject_id))
from noteevents
where category like 'Echo%'; 
"""
data = pd.read_sql_query(query, con)
data

,count
0,22316


In [17]:
# Radiology reports
query = \
"""
select count(distinct(subject_id))
from noteevents
where category like 'Radiology%'; 
"""
data = pd.read_sql_query(query, con)
data

,count
0,37351


### Now query Diabetes `structured` data

<b>Diabetes types in ICD9 code</b>
* Diabetes mellitus
    * 249 secondary diabetes mellitus (includes the following codes: 249, 249.0, 249.00, 249.01, 249.1, 249.10, 249.11, 249.2, 249.20, 249.21, 249.3, 249.30, 249.31, 249.4, 249.40, 249.41, 249.5, 249.50, 249.51, 249.6, 249.60, 249.61, 249.7, 249.70, 249.71, 249.8, 249.80, 249.81, 249.9, 249.90, 249.91)

    * 250 diabetes mellitus (includes the following codes: 250, 250.0, 250.00, 250.01, 250.02, 250.03, 250.1, 250.10, 250.11, 250.12, 250.13, 250.2, 250.20, 250.21, 250.22, 250.23, 250.3, 250.30, 250.31, 250.32, 250.33, 250.4, 250.40, 250.41, 250.42, 250.43, 250.5, 250.50, 250.51, 250.52, 250.53, 250.6, 250.60, 250.61, 250.62, 250.63, 250.7, 250.70, 250.71, 250.72, 250.73, 250.8, 250.80, 250.81, 250.82, 250.83, 250.9, 250.90, 250.91, 250.92, 250.93)

* Hemodialysis
    - 585.6 end stage renal disease (requiring chronic dialysis)
    - 996.1 mechanical complication of other vascular device, implant, and graft
    - 996.73 other complications due to renal dialysis device, implant, and graft
    - E879.1 kidney dialysis as the cause of abnormal reaction of patient, or of later complication, without mention of misadventure at time of procedure
    - V45.1 postsurgical renal dialysis status
    - V56.0 encounter for extracorporeal dialysis
    - V56.1 fitting and adjustment of extracorporeal dialysis catheter
    
* Precedure codes
    - 38.95 venous catheterization for renal dialysis
    - 39.27 arteriovenostomy for renal dialysis
    - 39.42 revision of arteriovenous shunt for renal dialysis
    - 39.43 removal of arteriovenous shunt for renal dialysis
    - 39.95 hemodialysis

In [18]:
# Total number of patients in the Mimic III db is 46520, with 58976 records (someone has multiple records)
# Note: connecting to diagnoses_icd table won't impact the query result so it has full record as admissions
query = \
"""
select subject_id, hadm_id
from admissions; 
"""
data = pd.read_sql_query(query, con)
print('Total', data.shape[0], ' lines of record')
print('Totally', data.subject_id.unique().shape[0], 'unique patients')

Total 58976  lines of record
Totally 46520 unique patients


In [19]:
# Total number of patients who are Diabetes mellitus is 10403
# Note: connecting to patients table won't impact the query result so it has full records
query = \
"""
select count(distinct(a.subject_id))
from diagnoses_icd di, admissions a
where di.subject_id = a.subject_id
	and (
		di.ICD9_CODE like '249%'      -- secondary diabetes mellitus
		or di.ICD9_CODE like '250%'   -- diabetes mellitus 
		);
"""
data = pd.read_sql_query(query, con)
data

,count
0,10403


In [20]:
# Total number of patients who are Diabetes mellitus and older than 18 is 10397
query = \
"""
select count(distinct(a.subject_id))
from diagnoses_icd di, admissions a, patients p
where di.subject_id = a.subject_id and a.subject_id = p.subject_id
	and (
		di.ICD9_CODE like '249%'      -- secondary diabetes mellitus
		or di.ICD9_CODE like '250%'   -- diabetes mellitus 
		)
	and (
		(cast(a.ADMITTIME as date) - cast(p.DOB as date))/365.242 >= 18
		);
"""
data = pd.read_sql_query(query, con)
data

,count
0,10397


In [21]:
# Total number of patients who are Diabetes mellitus, older than 18, and also received procedures is 9460
query = \
"""
select count(distinct(a.subject_id))
from diagnoses_icd di, admissions a, patients p, procedures_icd pi
where di.subject_id = a.subject_id 
    and a.subject_id = p.subject_id   -- we have patient here for p.DOB information
    and pi.subject_id = a.subject_id
	and (
		di.ICD9_CODE like '249%'      -- secondary diabetes mellitus
		or di.ICD9_CODE like '250%'   -- diabetes mellitus 
		)
	and (
		(cast(a.ADMITTIME as date) - cast(p.DOB as date))/365.242 >= 18
		);
"""
data = pd.read_sql_query(query, con)
data

,count
0,9460


In [22]:
# Compared to all patients who received procdures as Hemodialysis is 1316
query = \
"""
select count(distinct(subject_id))
from diagnoses_icd
where ICD9_CODE in ('5856','9961','99673','E8791','V451','V560','V561');
"""
data = pd.read_sql_query(query, con)
data

,count
0,1316


In [25]:
# Patients who are Diabetes mellitus, older than 18, and received procedures as Hemodialysis is 718
# Note: diagnoses_icd also include Hemodialysis ICD9 code
# We want patients who have both Diabetes mellitus amd Hemodialysis, e.g. di.ICD9_CODE of one patient have both '249x' and '5856'
query = \
"""
with diab as (
	select distinct(a.subject_id) 			-- second diabtes adults who under procedures is 9460
	from diagnoses_icd di, admissions a, patients p, procedures_icd pi
	where di.subject_id = a.subject_id 
		and a.subject_id = p.subject_id		-- we have patient here for p.DOB information
		and pi.subject_id = a.subject_id
		and (
			di.ICD9_CODE like '249%'      	-- secondary diabetes mellitus
			or di.ICD9_CODE like '250%'   	-- diabetes mellitus 
			)
		and ((cast(a.ADMITTIME as date) - cast(p.DOB as date))/365.242 >= 18)  -- adults
		) 
select distinct(di.subject_id)  		-- second diabetes adults under hemodialysis procedures is 718
from diagnoses_icd di, diab
where di.subject_id = diab.subject_id
	and di.ICD9_CODE in ('5856','9961','99673','E8791','V451','V560','V561');  -- Hemodialysis
"""
data = pd.read_sql_query(query, con)
print('There are ', len(data), 'patients with diabetes mellitus, adults and received Hemodialysis')

There are  718 patients with diabetes mellitus, adults and received Hemodialysis


Remember to close the thread after all queries

In [8]:
cur.close()
print('cursor still open ...' if not cur.closed else 'cursor closed ...')
del cur
print('cursor deleted from instance ...')
con.close()
print('connection still open ...' if not con.closed else 'connection closed')

cursor closed ...
cursor deleted from instance ...
connection closed
